In [1]:
%load_ext autoreload
%autoreload 2

import gc
import time
import feather
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append('/home/chamhaolee/code/kaggle-R/Tools/')
from tools import *
from plot import *

In [2]:
fulldata = pd.concat([feather.read_dataframe('/home/chamhaolee/tempfile/R/train_v2_convert.feather'), 
                      feather.read_dataframe('/home/chamhaolee/tempfile/R/test_v2_convert.feather')], 
                     ignore_index=True).sort_values(['fullVisitorId','visitStartTime'])

In [3]:
print(f'fulldata shape: {fulldata.shape}')

fulldata shape: (2109926, 147)


这个 EDA ，目的是来根据当次的会话数据来预测其是否会在一定时间内回访。
* * *
为了这个目的，首先建立一个特征followup，来找出每次会话之后，下次会话间隔的时间，如果之后再也没有访问，则为NaN。

In [4]:
# 首先创建一个时间的feature，用来记录本次访问的日期
fulldata['date'] = pd.to_datetime(fulldata.date,format='%Y%m%d')

In [5]:
# eval visitNumber
fulldata['visitNumber'] = fulldata.visitNumber.apply(strtolist)
fulldata['visitStartTime'] = fulldata.visitStartTime.apply(strtolist)


In [6]:
def diffdate(visitNumber, date, i):
    try:
        if visitNumber[i+1] > visitNumber[i]:
            diff = abs((date[i+1] - date[i]).days)
        else:
            diff = np.nan
    except:
        diff = np.nan
    return diff

In [8]:
number = fulldata.visitNumber.tolist()
date = fulldata.date.tolist()
fulldata['difftime'] = [diffdate(number, date, i) for i in np.arange(fulldata.shape[0])]
del number
del date